In [1]:
########################################################################
# File       : Read_and_Display_Images_using_Widgets_and_Napari.ipynb
# Version    : 0.1
# Author     : czsrh
# Date       : 26.11.2019
# Institution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
########################################################################

***Import Libaries to run this notebook***

* [PyPi - czifile](https://pypi.org/project/czifile/)

Thanks to Christoph Gohlke. For details and the source code please see [czifile.py](https://www.lfd.uci.edu/~gohlke/code/czifile.py.html)

* [PyPi - apeer-ometiff-library](https://pypi.org/project/apeer-ometiff-library/)

It uses parts of python-bioformats library: [PyPi - python-bioformats](https://pypi.org/project/python-bioformats/).

More information on the source code can be found on the APEER GitHub project page: [GitHub - apeer-ometiff-library](https://github.com/apeer-micro/apeer-ometiff-library)

* [PyPi - napari](https://pypi.org/project/napari/)

Very new, fast and cool multi-dimensional array viewer. For more details see the prject page [GitHub - napari](https://github.com/napari/napari)

* imagefileutils

This is collection of of usefule function to read CZI and OME-TFF image files. This tool uses many of the modules mentioned above

***

Another important modules

* [PyPi - xmltodict](https://pypi.org/project/xmltodict/)

This is turns an XML, e.g. the OME-XML or CZI metadata into an "easy-to-use" python dictionary.

In [2]:
# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import sys
sys.path.append(r'modules')
import imgfileutils as imf

# import the libraries mentioned above
from apeer_ometiff_library import io, processing, omexmlClass
import czifile as zis
import xmltodict
import os
import numpy as np
import ipywidgets as widgets
import napari
import xml.etree.ElementTree as ET

In [3]:
# define your testfiles here

#filename = 'Nucleus_Mito.czi'
filename = 'CellDivision_T=10_Z=15_CH=2_DCV_small.czi'
#filename = '3CH_Z-Stack_RN-DAPI.ome.tiff'

In [4]:
# !!! Only use for CZI files !!! 

# To read the CZI metadata one can use the czifile functionality

# get czi object using the filename
czi = zis.CziFile(filename)

# get the metadata and close the czi file object
mdczi = czi.metadata()
czi.close()

# parse the XML into a dictionary
metadatadict_czi = xmltodict.parse(mdczi)

In [5]:
# !!! Only use for CZI files !!! 

# Sometime it can be very usefule to save the CZI metadate as an actual XML file.
# This can be done quite easily by converting the metadata string into an XML tree.
# This tree object can be directly as an XML file

# define the new filename for the XML to be created later
xmlfile = filename.replace('.czi', '_CZI_MetaData.xml')

# get the element tree
tree = ET.ElementTree(ET.fromstring(mdczi))

# write xml to disk
tree.write(xmlfile, encoding='utf-8', method='xml')

print('Write special CZI XML metainformation for: ', xmlfile)

Write special CZI XML metainformation for:  CellDivision_T=10_Z=15_CH=2_DCV_small_CZI_MetaData.xml


In [6]:
# !!! Only use for CZI files !!! 

# or much shorter
xmlczi = imf.writexml_czi(filename)
print(xmlczi)

CellDivision_T=10_Z=15_CH=2_DCV_small_CZI_MetaData.xml


### Remark

As one can clearly see there are a lot of metadata not all of them are needed for every workflow.

In [7]:
# read metadata and array differently for OME-TIFF or CZI data
if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata, add_metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):

    # get the array and the metadata
    array, metadata, add_metadata = imf.get_array_czi(filename, return_addmd=False)

In [8]:
# outout the shape of the returned numpy array

# shape of numpy array
print('Array Shape: ', array.shape)

# dimension order from metadata
print('Dimension Order (BioFormats) : ', metadata['DimOrder BF Array'])

# shape and dimension entry from CZI file as returned by czifile.py
print('CZI Array Shape : ', metadata['Shape'])
print('CZI Dimension Entry : ', metadata['Axes'])

Array Shape:  (1, 10, 2, 15, 256, 256)
Dimension Order (BioFormats) :  None
CZI Array Shape :  (1, 10, 2, 15, 256, 256, 1)
CZI Dimension Entry :  BTCZYX0


In [9]:
# show dimensions and scaling
print('SizeT : ', metadata['SizeT'])
print('SizeZ : ', metadata['SizeZ'])
print('SizeC : ', metadata['SizeC'])
print('SizeX : ', metadata['SizeX'])
print('SizeY : ', metadata['SizeY'])
print('XScale: ', metadata['XScale'])
print('YScale: ', metadata['YScale'])
print('ZScale: ', metadata['ZScale'])

SizeT :  10
SizeZ :  15
SizeC :  2
SizeX :  256
SizeY :  256
XScale:  0.091
YScale:  0.091
ZScale:  0.32


In [10]:
# show all the metadata
for key, value in metadata.items():
    # print all key-value pairs for the dictionary
    print(key, ' : ', value)

Directory  :  
Filename  :  CellDivision_T=10_Z=15_CH=2_DCV_small.czi
Extension  :  czi
ImageType  :  czi
Name  :  None
AcqDate  :  2016-02-12T09:41:02.4915604Z
TotalSeries  :  None
SizeX  :  256
SizeY  :  256
SizeZ  :  15
SizeC  :  2
SizeT  :  10
Sizes BF  :  None
DimOrder BF  :  None
DimOrder BF Array  :  None
DimOrder CZI  :  {'B': 0, 'S': -1, 'T': 1, 'C': 2, 'Z': 3, 'Y': 4, 'X': 5, '0': 6}
Axes  :  BTCZYX0
Shape  :  (1, 10, 2, 15, 256, 256, 1)
isRGB  :  None
ObjNA  :  1.2
ObjMag  :  50.0
ObjID  :  Objective:1
ObjName  :  Plan-Apochromat 50x/1.2
ObjImmersion  :  Water
XScale  :  0.091
YScale  :  0.091
ZScale  :  0.32
XScaleUnit  :  µm
YScaleUnit  :  µm
ZScaleUnit  :  µm
DetectorModel  :  Axiocam 506
DetectorName  :  Axiocam506m
DetectorID  :  Detector:Axiocam 506
InstrumentID  :  None
Channels  :  ['AF555', 'AF488']
ImageIDs  :  []
NumPy.dtype  :  uint16
PixelType  :  Gray16
SizeM  :  1
SizeB  :  1
SizeS  :  1
SW-Name  :  ZEN 3.1 (blue edition)
SW-Version  :  3.1.0.0000
TubelensMag 

In [11]:
# display data using ipy widgets
if metadata['Extension'] == 'ome.tiff':
    ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
if metadata['Extension'] == 'czi':
    ui, out = imf.create_ipyviewer_czi(array, metadata)

display(ui, out)

0 29801


Output()

In [12]:
# try to configure napari automatiaclly based on metadata
imf.show_napari(array, metadata)

Initializing Napari Viewer ...
Dim PosT :  1
Dim PosZ :  3
Dim PosC :  2
Scale Factors :  [1.0, 1.0, 1.0, 3.516, 1.0, 1.0]
Shape Channel :  0 (1, 10, 15, 256, 256)
Adding Channel:  AF555
Scaling Factors:  [1.0, 1.0, 1.0, 3.516, 1.0, 1.0]
Scaling:  [0, 4927.0]
Shape Channel :  1 (1, 10, 15, 256, 256)
Adding Channel:  AF488
Scaling Factors:  [1.0, 1.0, 1.0, 3.516, 1.0, 1.0]
Scaling:  [0, 25331.0]


jupyter nbconvert MIAP_Read_and_Display_Images.ipynb --to slides --post serve